In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd 
import os
from sklearn.metrics import roc_curve, roc_auc_score
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from tensorflow import keras
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout,BatchNormalization,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Layer, MultiHeadAttention
from tensorflow.keras.layers import LayerNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LeakyReLU,LSTM, TimeDistributed
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc,accuracy_score
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
import requests
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import seaborn as sns
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix


In [ ]:
train_dir='/kaggle/input/dataset-splitted/splitted_dataset/Train'
validation_dir='/kaggle/input/dataset-splitted/splitted_dataset/Validation'
test_dir='/kaggle/input/dataset-splitted/splitted_dataset/Test'

In [ ]:
train_generator = ImageDataGenerator(
    rescale=1./255,                  
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
validation_generator = ImageDataGenerator(rescale=1./255)
test_generator = ImageDataGenerator(rescale=1./255)

train_flow = train_generator.flow_from_directory(train_dir, target_size=(224,224), batch_size=16, class_mode='binary')
validation_flow = validation_generator.flow_from_directory(validation_dir, target_size=(224,224), batch_size=16, class_mode='binary')
test_flow = test_generator.flow_from_directory(test_dir, shuffle= False , target_size=(224,224), batch_size=16, class_mode='binary')


In [ ]:
# Preparing the Inception ResNet V2 model
base_model = InceptionResNetV2(weights='imagenet', include_top=False)
for layer in base_model.layers:
    layer.trainable = True
# Add custom layers
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = TimeDistributed(Flatten())(x)
x = LSTM(128)(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs, outputs)
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.summary()

In [ ]:
checkpoint_path = '.\\tmp_checkpoint'
print('creating Directory: ' + checkpoint_path)
os.makedirs(checkpoint_path, exist_ok=True)

cus_callbacks = [
    EarlyStopping(
        monitor='loss',  # Monitor the loss value
        patience=5,  # Stop training when loss has not improved for 5 epochs
        # min_delta=0.001,  # Minimum change in loss to qualify as an improvement
        mode='min',  # Minimize the loss
       verbose=1
    ),
    ModelCheckpoint(
        filepath=os.path.join(checkpoint_path, 'best_model_ResnetV2.keras'),
        monitor='val_loss',  # Note the lowercase 'val_loss'
        mode='min',
        verbose= 1,
        save_best_only=True,
         
    )
]

In [ ]:
history=model.fit(train_flow, epochs=50, validation_data= validation_flow,callbacks=cus_callbacks)

In [ ]:
from tensorflow.keras.models import load_model
best_model=load_model(os.path.join(checkpoint_path,'best_model_ResnetV2.keras'))

loss, accuracy = best_model.evaluate(test_flow)
# Convert accuracy to percentage
accuracy_percentage = accuracy * 100

print(f'test accuracy: {accuracy_percentage:.2f}%')

In [ ]:
loss, accuracy = best_model.evaluate(validation_flow)
# Convert accuracy to percentage
accuracy_percentage = accuracy * 100

print(f'Validation accuracy: {accuracy_percentage:.2f}%')

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim(0.5, 1.1)
plt.xlim(1,23) 
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim(0,0.7)
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
# Get an image and its label from the test flow
img, label = next(test_flow)
label_ = label.argmax(axis=-1)

#make prediction using the model
res = model.predict(img)

# Determine the predicted class
class_ = res.argmax(axis=1)

In [ ]:
# Display the image
plt.imshow(img[1])
plt.show()

In [ ]:
if (class_ == 0).all():
    if (label_ == 0).all():
        print("Actual class is fake, predicted class is fake")
    else:
        print("Actual class is real, predicted class is fake")
else:
    if (label_ == 0).all():
        print("Actual class is fake, predicted class is real")
    else:
        print("Actual class is real, predicted class is real")

In [ ]:
test_pred = best_model.predict(test_flow)
test_pred_classes = np.round(test_pred)


# Get the true classes of the test set
test_true_classes = test_flow.classes

In [ ]:
names=['fake','real']
print("Confusion Matrix")
cm=confusion_matrix(test_flow.classes,test_pred_classes)
print(cm)

In [ ]:
#Plot confusion matrix as a heatmap
# Generate confusion matrix
import seaborn as sns
cm = confusion_matrix(test_true_classes, test_pred_classes)
plt.figure(figsize=(4,2))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=test_flow.class_indices.keys(),
            yticklabels=test_flow.class_indices.keys())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
#Classification report
report = classification_report(
    test_true_classes,
    test_pred_classes, 
    target_names=["fake", "real"],
    labels=[0, 1], digits=4)
print("Classification Report:\n", report)

In [ ]:
# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(test_true_classes, test_pred)

# Calculate AUC
auc = roc_auc_score(test_true_classes, test_pred)

# Plot ROC curve
plt.figure(figsize=(6, 4))
plt.plot(fpr, tpr, label='ROC curve (AUC = {:.4f})'.format(auc))
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

In [ ]:
tn, fp, fn, tp = confusion_matrix(test_true_classes, test_pred_classes).ravel()

# Calculate sensitivity and specificity
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print(f'Sensitivity (True Positive Rate): {sensitivity:.4f}')
print(f'Specificity (True Negative Rate): {specificity:.4f}')

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

# Assuming you have obtained test_pred_classes and test_true_classes as described earlier

# Calculate F1 score for the test classes
f1 = f1_score(test_true_classes, test_pred_classes)
# Calculate accuracy to get error rate
accuracy = accuracy_score(test_true_classes, test_pred_classes)
error_rate = 1 - accuracy
# Calculate G-mean
g_mean = (sensitivity * specificity) ** 0.5

# Calculate F-measure
f_measure = 2 * ((sensitivity * specificity) / (sensitivity + specificity))

print(f'F1 Score: {f1:.4f}')
print(f'Error Rate: {error_rate:.4f}')
print(f'G-mean: {g_mean:.4f}')
print(f'F-measure: {f_measure:.4f}')

In [ ]:
# Save the model
model_save_path = "ResnetV2_with_LSTM_deepfake_detection_model.h5"
model.save(model_save_path)
print("Model saved at:", model_save_path)